In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import lightgbm

### Data

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
international_trade = pd.read_csv("../data/international_trade.csv")

In [3]:
# for time series model
train["case"] = train["ID"].apply(lambda x : x[0:6])

## for machine learning model
train["timestamp"] = pd.to_datetime(train["timestamp"])
train["year"] = train["timestamp"].dt.year
train["month"] = train["timestamp"].dt.month
train["day"] = train["timestamp"].dt.day
train = train.drop(columns=["ID", "supply(kg)"])

X_train = train[["case", "timestamp", "item", "corporation", "location", "year", "month", "day"]]
y_train = train["price(원/kg)"]

In [4]:
## OneHotEncoding
for catfeature in ["item", "corporation", "location"]:
    encoder = OneHotEncoder(sparse=False)
    fitted_encoder = encoder.fit(X_train[[catfeature]]) 
    cat = pd.DataFrame(fitted_encoder.transform(X_train[[catfeature]].to_numpy()), columns=[str(col) for col in fitted_encoder.categories_[0]])
    X_train = X_train.drop(columns=catfeature)
    X_train = pd.concat([X_train, cat], axis=1)

### Time Series Split

In [144]:
def walkforward(X_by_case, y_by_case, n_splits:int, max_train_size:int):        
    '''
    walk forward 의 진행 상태를 보여주는 함수입니다. 
    기존 값은 black, 학습에 사용된 데이터는 blue, 검증에 사용된 데이터는 green, 모델에 의해 예측된 데이터는 red로 표시하였습니다.
        Args:
        X: 한 case의 X train data set
        y: 한 case의 y train data set
        limit:
            True: training data set의 수를 제한합니다.
            False: trainig data set이 계속해서 증가합니다. 
        train_size: limit가 True인 경우 train_size를 설정해야합니다. train_size를 설정하지 않을 경우 limit가 False 인 경우와 동일합니다.
    '''

    use_features = list(X_by_case.drop(columns=["case", "timestamp"]).columns)
    scores = []
    walkforward = TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size)
    
    for i, (index_train, index_valid) in enumerate(walkforward.split(X_by_case)):
        print(f"Walk Forward: {i}")
        print(index_train)
        
        X_tr, y_tr = X_by_case.loc[index_train, use_features], y_by_case.loc[index_train]
        X_val, y_val = X_by_case.loc[index_valid, use_features], y_by_case.loc[index_valid]

        model = lightgbm.LGBMRegressor()
        model.fit(X_tr, y_tr, eval_metric="rmse", eval_set=[(X_tr, y_tr), (X_val, y_val)], verbose=False)
        y_val_pred = model.predict(X_val)

        each_step_rmse = mean_squared_error(y_true=y_val, y_pred=y_val_pred, squared=False)
        scores.append(each_step_rmse)
        print(f"     Train Set len: {len(index_train)}")
        print(f"     Validation Set len: {len(index_valid)}")
        print(f"     Rmse: {each_step_rmse}")
        
    score = np.mean(scores)
    print("-----------------------------------")
    print(f"Mean Rmse: {score}")
    
    return score 


In [145]:
def walkforward_process(X, y, n_splits:int, max_train_size:int=None):
    cases = list(X["case"].unique())
    case_scores = {}
    
    for i, case_name in enumerate(cases): 
        case_index = list(X[X["case"] == case_name].index)
        X_by_case = X_train.loc[case_index, :].reset_index(drop=True)
        y_by_case = y_train.loc[case_index].reset_index(drop=True)
        # max_train_size = len(X_by_case) // (n_splits + 1) + len(X_by_case) % (n_splits + 1)

        # max_train_size=200
        score = walkforward(X_by_case=X_by_case, y_by_case=y_by_case, n_splits=n_splits, max_train_size=max_train_size)
        case_scores[f"{case_name}"] = score
    
    mean_score = np.mean(list(case_scores.values()))
    
    return mean_score

In [149]:
1573//5

314

In [167]:
len(y_by_case) // 5

304

In [172]:
def custom_walkforwad_by_case(X, y, n_splits, valid_size):
    train_size = len(y) // n_splits
    
    total_y_val = []
    total_y_val_pred = []
    
    for i in range(n_splits):
        s_block, e_block = i * train_size, (i+1) * train_size
        
        if i == n_splits:
            valid_size = len(y) - e_block
    
        X_tr, y_tr = X[s_block:e_block], y[s_block:e_block]
        X_val, y_val = X[e_block:e_block + valid_size], y[e_block:e_block + valid_size]

        model = lightgbm.LGBMRegressor()
        model.fit(X_tr, y_tr, eval_metric="rmse", eval_set=[(X_tr, y_tr), (X_val, y_val)], verbose=False)
        y_val_pred = model.predict(X_val)
        
        total_y_val.append(y_val)
        total_y_val_pred.append(y_val_pred)
    
    return total_y_val, total_y_val_pred

In [170]:
def custom_walkforwad_increasing_by_case(X, y, n_splits, valid_size):
    train_size = len(y) // n_splits
    
    total_y_val = []
    total_y_val_pred = []
    
    for i in range(n_splits):
        e_block = (i+1) * train_size
        
        if i == n_splits:
            valid_size = len(y) - e_block

        X_tr, y_tr = X[0:e_block], y[0:e_block]
        X_val, y_val = X[e_block:e_block + valid_size], y[e_block:e_block + valid_size]

        model = lightgbm.LGBMRegressor()
        model.fit(X_tr, y_tr, eval_metric="rmse", eval_set=[(X_tr, y_tr), (X_val, y_val)], verbose=False)
        y_val_pred = model.predict(X_val)
        
        total_y_val.append(y_val)
        total_y_val_pred.append(y_val_pred)
    
    return total_y_val, total_y_val_pred

In [171]:
cases = list(X_train["case"].unique())
for case_name in cases: 
    
    case_index = list(X_train[X_train["case"] == case_name].index)
    X_by_case = X_train.loc[case_index, :]
    y_by_case = y_train.loc[case_index]    
    
    custom_walkforwad_increasing_by_case(X=X_by_case, y=y_by_case, n_splits=5, valid_size=28)

-----
train_X index : Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       294, 295, 296, 297, 298, 299, 300, 301, 302, 303],
      dtype='int64', length=304)
valid_X index : Index([304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317,
       318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331],
      dtype='int64')
train_X index : Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       598, 599, 600, 601, 602, 603, 604, 605, 606, 607],
      dtype='int64', length=608)
valid_X index : Index([608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621,
       622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635],
      dtype='int64')
train_X index : Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       902, 903, 904, 905, 906, 907, 908, 909, 910, 911],
      dtype='int64', length=912)
valid_X index : Index([912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 92

In [146]:
score_dict = walkforward_process(X=X_train, y=y_train, n_splits=5)
# score_dict = walkforward_process(X=X_train, y=y_train, n_splits=5) # increasing

Walk Forward: 0
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 24

750.2156473421178

### Visualization

In [42]:
# def walkforward(X_by_case, y_by_case, n_splits:int, limit:bool, train_size:int=None, visualization:bool=False):        
#     '''
#     walk forward 의 진행 상태를 보여주는 함수입니다. 
#     기존 값은 black, 학습에 사용된 데이터는 blue, 검증에 사용된 데이터는 green, 모델에 의해 예측된 데이터는 red로 표시하였습니다.
#         Args:
#         X: 한 case의 X train data set
#         y: 한 case의 y train data set
#         limit:
#             True: training data set의 수를 제한합니다.
#             False: trainig data set이 계속해서 증가합니다. 
#         train_size: limit가 True인 경우 train_size를 설정해야합니다. train_size를 설정하지 않을 경우 limit가 False 인 경우와 동일합니다.
#     '''

#     use_features = list(X.columns)[2:]
#     case_name = list(X['case'].unique())[0]
#     scores = []
    
#     if limit:
#         walkforward = TimeSeriesSplit(n_splits=n_splits, max_train_size=train_size)
#     else:
#         walkforward = TimeSeriesSplit(n_splits=n_splits)

#     if visualization:
#         plt.figure(figsize=(12, 5))
#         sns.scatterplot(x=X["timestamp"], y=y, marker=".", color="black", alpha=0.5)
#         plt.title(f"{case_name}")
#         plt.show()

#     for i, (index_train, index_valid) in enumerate(walkforward.split(X)):
#         print(f"Walk Forward: {i}")
#         X_tr, y_tr = X.loc[index_train, use_features], y.loc[index_train]
#         X_val, y_val = X.loc[index_valid, use_features], y.loc[index_valid]
#         X_tr_timestamp, X_val_timestamp = X.loc[index_train, "timestamp"], X.loc[index_valid, "timestamp"]

#         model = lightgbm.LGBMRegressor()
#         model.fit(X_tr, y_tr, eval_metric="rmse", eval_set=[(X_tr, y_tr), (X_val, y_val)], verbose=False)
#         y_val_pred = model.predict(X_val)

#         if visualization:
#             plt.figure(figsize=(12, 5))
#             sns.scatterplot(x=X["timestamp"], y=y, marker=".", color="black", alpha=0.5)
#             sns.scatterplot(x=X_tr_timestamp, y=y_tr, marker=".", color="blue", alpha=0.5)
#             sns.scatterplot(x=X_val_timestamp, y=y_val, marker=".", color="green", alpha=0.5)    
#             sns.scatterplot(x=X_val_timestamp, y=y_val_pred, marker="o", color="red", alpha=0.5)    
            
#             plt.axvline(x=X_val_timestamp.head(1), color="red")
#             plt.axvline(x=X_val_timestamp.tail(1), color="red")
            
#             plt.title(f"Walk Forward {i+1}: {case_name}")
#             plt.show()

#         each_step_rmse = mean_squared_error(y_true=y_val, y_pred=y_val_pred, squared=False)
#         scores.append(each_step_rmse)
#         print(f"     Train Set len: {len(index_train)}")
#         print(f"     Validation Set len: {len(index_valid)}")
#         print(f"     Rmse: {each_step_rmse}")
        
#     score = np.mean(scores)
#     print("-----------------------------------")
#     print(f"Mean Rmse: {score}")
    
#     return score 
